In [0]:
# Setting Catalog and Schema
catalog_name = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema")
volume_name = "raw"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Location of the raw data
location_raw = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/"


In [0]:
# GET file_jsonl and folder_volume param's
file_jsonl = dbutils.widgets.get("file_jsonl")
folder_volume = dbutils.widgets.get("folder_volume")

# # TESTE
# file_jsonl = "./output/purchaseorderdetail.jsonl"
# folder_volume = "api_meltano"

# Setting prefix_table
prefix_table = 'raw_api_'


table_name = (prefix_table + 
              (file_jsonl.replace("./output/","")
              [:(file_jsonl.replace("./output/","")
                 .find("."))]
              .lower()))


print(table_name)

file_jsonl = f'{location_raw}{folder_volume}/{file_jsonl.replace("./output/","")}'



bronze_api_purchaseorderdetail


In [0]:
try:
    sql_command = (f'''
                   COPY INTO {table_name}
                   FROM '{file_jsonl}'
                   FILEFORMAT = JSONL
                   ''')
    spark.sql(sql_command)
except Exception as e:
    df = spark.read.json(file_jsonl)
    df.write.format("delta").mode("overwrite").saveAsTable(f"{table_name}")
